# undersanding how to correct perspective

+ [link](https://dev.intelrealsense.com/docs/projection-texture-mapping-and-occlusion-with-intel-realsense-depth-cameras)

In [1]:
import numpy as np
import cv2

In [3]:
def depth_to_point_cloud(depth_image, rgb_image, intrinsics):
    height, width = depth_image.shape

    # Parametri intrinseci
    f_x, f_y = intrinsics['f_x'], intrinsics['f_y']
    c_x, c_y = intrinsics['c_x'], intrinsics['c_y']

    # Crea una meshgrid delle coordinate dei pixel
    u, v = np.meshgrid(np.arange(width), np.arange(height))

    # print(u.shape)
    # print(v.shape)



    # Calcola le coordinate 3D
    X = (u - c_x) * depth_image / f_x
    Y = (v - c_y) * depth_image / f_y
    Z = depth_image


    # print(X.shape)
    # print(Y.shape)
    # print(Z.shape)
    # print(Z.size)
    



    # Impila le coordinate X, Y, Z
    points = np.stack((X, Y, Z), axis=-1).reshape(-1, 3)

    # print(points.shape)

    # Ottieni i valori RGB
    rgb_values = rgb_image.reshape(-1, 3)

    # Combina coordinate e colori in un unico array
    point_cloud = np.hstack((points, rgb_values))


    # reshape the data to have x, y values to navigate the channel values:
    # -  x[mm] (with respect to center of the frame)
    # -  y[mm]
    # -  z[mm]
    # -  red
    # -  green
    # -  blue
    point_cloud2 = point_cloud.reshape((depth_image.shape[0], depth_image.shape[1], len(point_cloud[0]) ))
  
    return point_cloud2

In [4]:
col3D = '../data/3D/1_1_6_0/color_1_6_0.png'
depth3D = '../data/3D/1_1_6_0/depth_1_6_0.png'


col3D = cv2.imread(col3D)
depth3D = cv2.imread(depth3D, cv2.IMREAD_ANYDEPTH)

# matrice K is intrinsics - per proiettare il pixel in punto xyz
intrinsics = {'f_x': 322.282410, 'f_y': 322.282410, 'c_x': 320.818268, 'c_y': 178.779297}


point_cloud = depth_to_point_cloud(depth3D, col3D, intrinsics)
print(point_cloud.shape)


bboxF = np.array([[37, 160], [104, 235]])
bboxT = np.array([[546,1770], [ 575,1871]])
print(np.mean(point_cloud[bboxF[0][1] : bboxF[1][1], bboxF[0][0] : bboxF[1][0], 0]))
print(np.mean(point_cloud[bboxT[0][1] : bboxT[1][1], bboxT[0][0] : bboxT[1][0], 0]))




(1920, 1080, 6)
-1133.6844084616955
1146.1436823124805
